In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder, StandardScaler 
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
#from lightgbm import LGBMClassifier
#from category_encoders import OneHotEncoder
from sklearn.model_selection import cross_val_predict
from warnings import filterwarnings
from sklearn.impute import SimpleImputer
filterwarnings('ignore')
import os
print(os.listdir("../input"))

input_path = ""
output_path = ""
train = pd.read_csv("data/house price/train.csv")
print("train shape", train.shape)
test = pd.read_csv("data/house price/test.csv")
print("test shape", test.shape)

target_column = "target"
id_column = "id"
categorical_cols = [c for c in test.columns if test[c].dtype in [np.object]]
numerical_cols = [c for c in test.columns if test[c].dtype in [np.float, np.int] and c not in [target_column, id_column]]
print("Number of features", len(categorical_cols)+len(numerical_cols))

# lightGBM

"""classifier = make_pipeline(
    ColumnTransformer([('num', StandardScaler(), numerical_cols), ('cat', OneHotEncoder(), categorical_cols),]), \
        LGBMClassifier(n_jobs=-1))

oof_pred = cross_val_predict(classifier, train, train[target_column], cv=5, method="predict_proba")"""

ImportError: dlopen(/opt/homebrew/Caskroom/miniconda/base/envs/py38/lib/python3.8/site-packages/scipy/sparse/linalg/_isolve/_iterative.cpython-38-darwin.so, 0x0002): Library not loaded: @rpath/liblapack.3.dylib
  Referenced from: /opt/homebrew/Caskroom/miniconda/base/envs/py38/lib/python3.8/site-packages/scipy/sparse/linalg/_isolve/_iterative.cpython-38-darwin.so
  Reason: tried: '/opt/homebrew/Caskroom/miniconda/base/envs/py38/lib/python3.8/site-packages/scipy/sparse/linalg/_isolve/liblapack.3.dylib' (no such file), '/opt/homebrew/Caskroom/miniconda/base/envs/py38/lib/python3.8/site-packages/scipy/sparse/linalg/_isolve/../../../../../../liblapack.3.dylib' (no such file), '/opt/homebrew/Caskroom/miniconda/base/envs/py38/lib/python3.8/site-packages/scipy/sparse/linalg/_isolve/liblapack.3.dylib' (no such file), '/opt/homebrew/Caskroom/miniconda/base/envs/py38/lib/python3.8/site-packages/scipy/sparse/linalg/_isolve/../../../../../../liblapack.3.dylib' (no such file), '/opt/homebrew/Caskro

In [39]:
train = pd.read_csv("/Users/jiayixian/projects/coding/data/house price/train.csv")
print("train shape", train.shape)
test = pd.read_csv("/Users/jiayixian/projects/coding/data/house price/test.csv")
print("test shape", test.shape)

train shape (1460, 81)
test shape (1459, 80)


In [2]:
df = pd.read_csv(input_path)

NameError: name 'input_path' is not defined

In [5]:
# Get categorical features and quantitative features
quantitative = [f for f in train.columns if train.dtypes[f] != 'object']
categorical = [f for f in train.columns if train.dtypes[f] == 'object']

In [8]:
print(len(quantitative), len(categorical))

38 43


In [40]:
df_na = pd.DataFrame(train.isnull().sum())
nacols = df_na[df_na[0] != 0]
todrop = nacols[nacols[0]/len(train)>0.1]
todrop

,0
LotFrontage,259
Alley,1369
FireplaceQu,690
PoolQC,1453
Fence,1179
MiscFeature,1406


In [60]:
train = train.drop(list(todrop.index), axis = 1)

In [61]:
list(todrop.index)

['LotFrontage', 'Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']

In [62]:
nacols = nacols.drop(todrop.index, axis = 0)

In [63]:
train.head()

,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000


In [64]:
nacols

,0
MasVnrType,8
MasVnrArea,8
BsmtQual,37
BsmtCond,37
BsmtExposure,38
BsmtFinType1,37
BsmtFinType2,38
Electrical,1
GarageType,81
GarageYrBlt,81


In [65]:
list(nacols.index)

['MasVnrType',
 'MasVnrArea',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageQual',
 'GarageCond']

In [67]:
res = train
for feature in list(nacols.index):
    res = res.dropna(subset=[feature])

In [69]:
train = train.dropna(subset=list(nacols.index))

,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,8,2007,WD,Normal,175000
1456,1457,20,RL,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2010,WD,Normal,210000
1457,1458,70,RL,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,112,0,0,0,0,4,2010,WD,Normal,142125


In [68]:
res

,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,8,2007,WD,Normal,175000
1456,1457,20,RL,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2010,WD,Normal,210000
1457,1458,70,RL,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,112,0,0,0,0,4,2010,WD,Normal,142125


In [44]:
train

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [ ]:
# count conditions matching
df_bool = (df == 'CA')
df_bool.sum()
df_bool.sum(axis = 1)
df_bool.sum().sum()


In [ ]:
def different_ways_fillna():
    df.fillna(0)
    # Filling null value with the previous ones
    df.fillna(method ='pad')
    # Filling null value with the next ones 
    df.fillna(method ='bfill')
    # filling a null values using fillna()
    df["Gender"].fillna("No Gender", inplace = True)
    # will replace  Nan value in dataframe with value -99 
    df.replace(to_replace = np.nan, value = -99)
    # to interpolate the missing values
    df.interpolate(method ='linear', limit_direction ='forward')

In [ ]:

feature = ""
na_replace = ""
# total missing value in each columns
df.isnull().sum()

# creating bool series True for NaN values
bool_series = pd.isnull(df[feature])

# filling na
# filling a null values using fillna()
df[feature].fillna(na_replace, inplace = True)

: 

## drop values/ entries

In [ ]:
df.dropna(axis = 1)
df = df.dropna(axis = 0, how ='any')

: 

In [ ]:
dropna_cols = ["", ""]
df = df.dropna(subset=dropna_cols)

: 

In [ ]:
def dropna():
    df.dropna()
    # Dropping rows if all values in that row are missing. 
    df = df.dropna(how = 'all')
    # drop any entries when it is na in specified columns.
    df = df.dropna(subset=['name', 'toy'])
    # dropping columns if it has at least one nan value
    df = df.dropna(axis = 0, how ='any')
    df = df.dropna(axis = 1)

: 

# deal with imbalanced dataset

In [ ]:
# check data imbalance
df.Class.value_counts() # class -> label column
len(df.loc[df.Class==1]) / len(df.loc[df.Class == 0])

In [73]:
print(type(train.SaleCondition.value_counts()))
values = train.SaleCondition.value_counts()
unique = np.unique(train.SaleCondition.values)

<class 'pandas.core.series.Series'>


In [79]:
values.index

Index(['Normal', 'Partial', 'Abnorml', 'Family', 'Alloca', 'AdjLand'], dtype='object')

In [ ]:
for feature in values.index:
    if values[feature] < 100:
        feature_upsampled = resample(train[feature])

In [86]:
# deal with imbalance dataset (resampling)
majority = train[train['SaleCondition'] == 'Normal']
minority = train[train['SaleCondition'] == 'Abnormal']

df_majority = majority.sample(len(minority), random_state=0)
df_minority = minority.sample(len(majority), random_state=1)

### concat the majority and minority dataframes
#df = pd.concat([df_majority,minority])

## Shuffle the dataset to prevent the model from getting biased by similar samples
df = df.sample(frac=1, random_state=0)
"""from sklearn.utils import resample
# upsample minority
minority_upsampled = resample(minority,
                        replace=True, # sample with replacement
                        n_samples=len(majority), # match number in majority class
                        random_state=27) # reproducible results"""



ValueError: a must be greater than 0 unless no samples are taken

# analysis of correlation

In [87]:
df = train

In [108]:
import numpy as np

# Create correlation matrix
corr_matrix = df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# corr_matrix.where((np.triu(np.ones(corr_matrix.shape), k=1) + np.tril(np.ones(corr_matrix.shape), k=-1)).astype(bool))
# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

# Drop features 
df.drop(to_drop, axis=1, inplace=True)

/var/folders/lz/mnztjtn963qcybqd3svg_j4c0000gn/T/ipykernel_3523/368915494.py:7: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


In [106]:
for f in list(todrop.index):
    if f in categorical:
        categorical.remove(f)

In [107]:
for f in categorical:
    df[f] = pd.factorize(df[f])[0]

In [93]:
len(upper)

37

In [ ]:
from pandas import factorize

labels, categories = factorize(df["Light_Sensor_Status"])
df["labels"] = labels
abs(df["Light_Sensor_Reading"].corr(df["labels"]))

: 

### feature importance

In [ ]:
# plot feature importance manually
from numpy import loadtxt
from xgboost import XGBClassifier
from matplotlib import pyplot

# fit model no training data
model = XGBClassifier()
model.fit(X, y)
# feature importance
print(model.feature_importances_)
# plot
pyplot.bar(range(len(model.feature_importances_)), model.feature_importances_)
pyplot.show()

: 

# training

In [ ]:
from sklearn.model_selection import train_test_split
X = df.drop('Class', axis=1)
Y = df["Class"]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=27)

: 

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler 
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score


: 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=10).fit(X_train, y_train)
# predict on test set
rfc_pred = rfc.predict(X_test)

accuracy_score(y_test, rfc_pred)
f1_score(y_test, rfc_pred)
pd.DataFrame(confusion_matrix(y_test, rfc_pred))
recall_score(y_test, rfc_pred)

: 

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='liblinear').fit(X_train, y_train)
# predict on test set
lr_pred = lr.predict(X_test)

accuracy_score(y_test, lr_pred)
f1_score(y_test, lr_pred)
pd.DataFrame(confusion_matrix(y_test, lr_pred))
recall_score(y_test, lr_pred)

: 

In [ ]:
def datapipeline(path):

    df = pd.read_csv(path)
   

    # You can count missing values in each column by default, and in each row with axis=1
    df.isnull().sum()
    df.isnull().sum(axis = 1)
    df.count()
    df.count(axis=1)

    # check if there is at least one nan value
    df.isnull().values.sum() != 0
    # check if there is at least one not nan value
    df.isnull().values.sum() == df.size

    # For series
    s = df['state']
    print(s)
    # 0     NY
    # 1    NaN
    # 2     CA
    # Name: state, dtype: object

    print(s.isnull())
    # 0    False
    # 1     True
    # 2    False
    # Name: state, dtype: bool

    print(s.notnull())
    # 0     True
    # 1    False
    # 2     True
    # Name: state, dtype: bool

    print(s.isnull().any())
    # True

    print(s.isnull().all())
    # False

    print(s.isnull().sum())
    # 1

    print(s.count())
    # 2




    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LogisticRegression
    from sklearn.dummy import DummyClassifier
    from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score
    
    dummy = DummyClassifier(strategy='most_frequent').fit(X_train, y_train)
    dummy_pred = dummy.predict(X_test)
    accuracy_score(y_test, dummy_pred)


    # Modeling the data as is
    # Train model
    lr = LogisticRegression(solver='liblinear').fit(X_train, y_train)
    
    # Predict on training set
    lr_pred = lr.predict(X_test)
    # Checking accuracy
    accuracy_score(y_test, lr_pred)
    predictions = pd.DataFrame(lr_pred)
    predictions[0].value_counts()



    # train model
    rfc = RandomForestClassifier(n_estimators=10).fit(X_train, y_train)

    # predict on test set
    rfc_pred = rfc.predict(X_test)

    accuracy_score(y_test, rfc_pred)
    0.999592708069998
    # f1 score
    f1_score(y_test, rfc_pred)
    pd.DataFrame(confusion_matrix(y_test, rfc_pred))
    recall_score(y_test, rfc_pred)


    # deal with imbalance dataset (resampling)
    not_fraud = X[X.Class==0]
    fraud = X[X.Class==1]

    # upsample minority
    fraud_upsampled = resample(fraud,
                            replace=True, # sample with replacement
                            n_samples=len(not_fraud), # match number in majority class
                            random_state=27) # reproducible results

    # combine majority and upsampled minority
    upsampled = pd.concat([not_fraud, fraud_upsampled])

    # check new class counts
    upsampled.Class.value_counts()
    y_train = upsampled.Class
    X_train = upsampled.drop('Class', axis=1)

    upsampled = LogisticRegression(solver='liblinear').fit(X_train, y_train)

    upsampled_pred = upsampled.predict(X_test)

    # downsample majority
    # still using our separated classes fraud and not_fraud from above

    # downsample majority
    not_fraud_downsampled = resample(not_fraud,
                                    replace = False, # sample without replacement
                                    n_samples = len(fraud), # match minority n
                                    random_state = 27) # reproducible results

    # combine minority and downsampled majority
    downsampled = pd.concat([not_fraud_downsampled, fraud])

    # checking counts
    downsampled.Class.value_counts()

    """Method 1: Rename Specific Columns df. rename(columns = {'old_col1':'new_col1', 'old_col2':'new_col2'}, inplace = True)
    Method 2: Rename All Columns df. columns = ['new_col1', 'new_col2', 'new_col3', 'new_col4']
    Method 3: Replace Specific Characters in Columns df."""
    #Drop the rows where all elements are missing.

    df.dropna(how='all')

    # Keep only the rows with at least 2 non-NA values.

    df.dropna(thresh=2) 
    # drop columns: df.drop(['B', 'C'], axis=1), df.drop(columns=['B', 'C'])

    # drop duplicate
    df.drop_duplicates()
    df.drop_duplicates(subset=[''])
    #To remove duplicates and keep last occurrences, use keep.

    df.drop_duplicates(subset=['brand', 'style'], keep='last')

: 